In [781]:
import os
import uuid
from datetime import datetime, timedelta
from math import ceil
import numpy as np
import random
import pandas as pd
import scipy.stats as stats
from tqdm import tqdm

import plotly.graph_objects as go
import plotly.express as px

from sklearn.metrics import (
    explained_variance_score, max_error, mean_absolute_error,
    mean_squared_error, mean_squared_error, mean_squared_log_error,
    median_absolute_error, r2_score, mean_poisson_deviance, mean_gamma_deviance,
    mean_absolute_percentage_error
)

import warnings
warnings.filterwarnings('ignore')

In [2]:
DATA_PATH = '../data/raw/pinheiro'
PREDICTIONS_PATH = 'results'

In [3]:
data = pd.read_csv(os.path.join(DATA_PATH, 'data.csv'))
df_pred = pd.read_csv(os.path.join(PREDICTIONS_PATH, 'df_pred.csv'))

In [ ]:
# Baseline: 7 days moving average
data['unit_sales_baseline'] = data['unit_sales'].rolling(7).mean()

In [4]:
data.head()

,date,store_nbr,item_nbr,unit_sales,promotion_type,price,profit,stock,FREQUENCIAVISITA,PRAZOEENTREGA,...,EMBALAGEMCOMPRA,TIPOEMBALAGEMTRANSFERENCIA,EMBALAGEMTRANSFERENCIA,CODIGO_INTERNO,DESCRICAO,STATUS,type,SECAO,GRUPO,SUBGRUPO
0,2017-01-01,15,187,0.0,0,4.5,2.51,1.0,28,1,...,1,UN,1,187-2,CERV.HEINEKEN L.NECK,ATIVO,A,PADARIA,CHOPPERIA,CERVEJAS
1,2017-01-02,15,187,0.0,0,4.5,1.36,1.0,28,1,...,1,UN,1,187-2,CERV.HEINEKEN L.NECK,ATIVO,A,PADARIA,CHOPPERIA,CERVEJAS
2,2017-01-03,15,187,0.0,0,4.5,2.51,1.0,28,1,...,1,UN,1,187-2,CERV.HEINEKEN L.NECK,ATIVO,A,PADARIA,CHOPPERIA,CERVEJAS
3,2017-01-04,15,187,0.0,0,4.5,1.36,1.0,28,1,...,1,UN,1,187-2,CERV.HEINEKEN L.NECK,ATIVO,A,PADARIA,CHOPPERIA,CERVEJAS
4,2017-01-05,15,187,1.0,0,4.5,1.36,0.0,28,1,...,1,UN,1,187-2,CERV.HEINEKEN L.NECK,ATIVO,A,PADARIA,CHOPPERIA,CERVEJAS


In [5]:
df_pred.head()

,date,item_nbr,store_nbr,unit_sales_pred
0,2017-08-08,50181,12,20.084455
1,2017-08-09,50181,12,17.188959
2,2017-08-10,50181,12,19.350032
3,2017-08-11,50181,12,22.001130
4,2017-08-12,50181,12,26.054481


In [108]:
df_true_x_pred = data.merge(df_pred, on=['date', 'item_nbr', 'store_nbr'])[
    ['date', 'item_nbr', 'store_nbr', 'unit_sales', 'unit_sales_pred']
]

In [56]:
df = df_true_x_pred.groupby(['date'])[['unit_sales', 'unit_sales_pred']].sum().reset_index()

df_ = df[(df['date'] >= '2017-01-01') & (df['date'] < '2018-01-01')]

x = df_['date']
y_true = df_['unit_sales']
y_pred = df_['unit_sales_pred']

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y_true,
                    mode='lines',
                    name='y_true'))

fig.add_trace(go.Scatter(x=x, y=y_pred,
                    mode='lines',
                    name='y_pred'))

# fig.write_image("results/images/y_true_x_pred_2019.png")
fig.show()

---

In [119]:
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()
# scaler.fit(df_true_x_pred['unit_sales'].values.reshape(-1, 1))
# df_true_x_pred['unit_sales'] = scaler.transform(df_true_x_pred['unit_sales'].values.reshape(-1, 1))
# df_true_x_pred['unit_sales_pred'] = scaler.transform(df_true_x_pred['unit_sales_pred'].values.reshape(-1, 1))

In [62]:
metrics = [
    explained_variance_score, max_error, mean_absolute_error,
    mean_squared_error, mean_squared_error, mean_squared_log_error,
    median_absolute_error, r2_score, mean_poisson_deviance, mean_gamma_deviance,
    mean_absolute_percentage_error 
]

In [109]:
y_true = df_true_x_pred['unit_sales'].values.reshape(-1, 1)
y_pred = df_true_x_pred['unit_sales_pred'].values.reshape(-1, 1)

In [110]:
metric_names = []
metric_values = []
for metric in metrics:
    try:
        metric_values.append(metric(y_true, y_pred))
        metric_names.append(metric.__name__)
    except Exception as e:
        print(e)

Mean Tweedie deviance error with power=2 can only be used on strictly positive y and y_pred.


In [111]:
df_metric = pd.DataFrame({
    'metric': metric_names,
    'value': metric_values
})

# df_metric.to_csv('results/df_metric.csv', index=False)

In [112]:
df_metric

,metric,value
0,explained_variance_score,3.340352e-03
1,max_error,2.949578e+04
2,mean_absolute_error,3.262739e+01
3,mean_squared_error,4.486240e+04
4,mean_squared_error,4.486240e+04
5,mean_squared_log_error,3.409290e+00
6,median_absolute_error,2.007124e+01
7,r2_score,3.335244e-03
8,mean_poisson_deviance,8.651408e+01
9,mean_absolute_percentage_error,2.339375e+16


---

In [293]:
item_nbr_list = []
store_nbr_list = []
mdae_list = []

for item_nbr in df_true_x_pred['item_nbr'].unique():
    for store_nbr in df_true_x_pred['store_nbr'].unique():
        try:
            item_mask = df_true_x_pred['item_nbr'] == item_nbr
            store_mask = df_true_x_pred['store_nbr'] == store_nbr

            df = df_true_x_pred[item_mask & store_mask]

            y_true = df['unit_sales'].values.reshape(-1, 1)
            y_pred = df['unit_sales_pred'].values.reshape(-1, 1)

            mdae_list.append(median_absolute_error(y_true, y_pred))
            item_nbr_list.append(item_nbr)
            store_nbr_list.append(store_nbr)
        except Exception as e:
            print(e)

Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1

Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1

Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1

Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1

Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1

...

Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1

Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1

Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1

Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1

Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1

In [292]:
np.quantile(df_mdae['median_absolute_error'], 0.5)

20.38248202566188

In [303]:
df_mdae = pd.DataFrame({
    'item_nbr': item_nbr_list,
    'store_nbr': store_nbr_list,
    'median_absolute_error': mdae_list
})

      
df_mdae['performance'] = df_mdae['median_absolute_error'].apply(
    lambda x: 'high' if x < np.quantile(
        df_mdae['median_absolute_error'], 0.5
    ) else 'low'
)

# df_mdae.to_csv('results/df_mdae.csv', index=False)

In [304]:
df_mdae['median_absolute_error'].mean(), df_mdae['median_absolute_error'].std()

(22.156673328214644, 11.45315407383758)

In [305]:
df_mdae.sort_values(
    by='median_absolute_error',
    inplace=True
)

fig = px.histogram(
    df_mdae, x="median_absolute_error", 
    color='performance',
    color_discrete_map = {
        'low':'red',
        'high':'green',
    }
)
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.75)
# fig.write_image("results/images/mdae_histogram.png")
fig.show()

---

In [306]:
df_baseline = data[[
    'date', 'item_nbr', 'store_nbr', 
    'unit_sales', 'unit_sales_baseline'
]]

df_baseline = df_baseline.merge(
    df_true_x_pred[['date', 'item_nbr', 'store_nbr']],
    on = ['date', 'item_nbr', 'store_nbr']
)

In [307]:
item_nbr_list = []
store_nbr_list = []
mdae_list = []

for item_nbr in df_baseline['item_nbr'].unique():
    for store_nbr in df_baseline['store_nbr'].unique():
        try:
            item_mask = df_baseline['item_nbr'] == item_nbr
            store_mask = df_baseline['store_nbr'] == store_nbr

            df = df_baseline[item_mask & store_mask]

            y_true = df['unit_sales'].values.reshape(-1, 1)
            y_pred = df['unit_sales_baseline'].values.reshape(-1, 1)

            mdae_list.append(median_absolute_error(y_true, y_pred))
            item_nbr_list.append(item_nbr)
            store_nbr_list.append(store_nbr)
        except Exception as e:
            print(e)

Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1

Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1

Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1

Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1

Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1

...

Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1

Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1

Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1

Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1

Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1

In [311]:
df_mdae_baseline = pd.DataFrame({
    'item_nbr': item_nbr_list,
    'store_nbr': store_nbr_list,
    'median_absolute_error': mdae_list
})


df_mdae_baseline['complexity'] = df_mdae_baseline['median_absolute_error'].apply(
    lambda x: 'low' if x < np.quantile(
        df_mdae['median_absolute_error'], 0.5
    ) else 'high'
)

# df_mdae_baseline.to_csv('results/df_mdae_baseline.csv', index=False)

In [312]:
df_mdae_baseline['median_absolute_error'].mean(), df_mdae_baseline['median_absolute_error'].std()

(6.824318181818172, 9.713225652011701)

In [313]:
df_mdae_baseline.sort_values(
    by='median_absolute_error',
    inplace=True
)

fig = px.histogram(
    df_mdae_baseline, x="median_absolute_error", 
    color='complexity',
    color_discrete_map = {
        'low':'green',
        'high':'red',
    }
)
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.75)
# fig.write_image("results/images/mdae_baseline_histogram.png")
fig.show()

In [703]:
df_classification = df_mdae.merge(
    df_mdae_baseline,
    on=['item_nbr', 'store_nbr']
)[['item_nbr', 'store_nbr', 'complexity', 'performance']]

In [704]:
df_classification

,item_nbr,store_nbr,complexity,performance
0,3017,12,low,high
1,1621,12,low,high
2,1632,12,low,high
3,1857,12,low,high
4,12297,12,low,high
...,...,...,...,...
237,60504,12,high,low
238,10315,12,low,low
239,111331,12,high,low
240,17183,12,low,low


In [705]:
df_classification.groupby(['complexity', 'performance']).count().reset_index()

,complexity,performance,item_nbr,store_nbr
0,high,high,3,3
1,high,low,13,13
2,low,high,118,118
3,low,low,108,108


In [316]:
# df_classification.to_csv('results/classification.csv', index=False)

---

In [721]:
df_experiments = df_classification.append(
    [df_classification]*4,
    ignore_index=True
)

df_experiments.rename(
    columns={
        'complexity': 'problem_complexity',
        'performance': 'model_performance'
    },
    inplace=True
)

complexities = ['high', 'low']
df_experiments['input_complexity'] = random.choices(
    complexities, 
    k=len(df_experiments)
)

df_experiments['id'] = [uuid.uuid4() for _ in range(len(df_experiments))]

first_column = df_experiments.pop('id')
df_experiments.insert(0, 'id', first_column)

# df_experiments.to_csv('results/experiments.csv', index=False)

In [725]:
df_experiments

,id,item_nbr,store_nbr,problem_complexity,model_performance,input_complexity
0,db7dc6f6-76f2-423d-bfd1-bd8a5c6d2bb8,3017,12,low,high,high
1,829b4f49-918c-4fdf-bf6c-93fb024e1457,1621,12,low,high,high
2,f913bafd-2008-401e-a473-1c3221c6f5b1,1632,12,low,high,low
3,a30e034b-7f8f-426c-8e4d-412f25c417b9,1857,12,low,high,high
4,38337c85-43ec-4d91-af21-3c3f0d2c1c92,12297,12,low,high,low
...,...,...,...,...,...,...
1205,48ec4dc0-34ea-4ce4-9452-507509229402,60504,12,high,low,low
1206,1db0c120-0fbf-4198-a394-0c01a4937b36,10315,12,low,low,high
1207,5321c8e2-5217-45c8-b612-e61c1cfb1a00,111331,12,high,low,low
1208,0248de10-9d33-4519-8208-cff5c9cca35d,17183,12,low,low,high


In [652]:
avg_data = data.groupby(['item_nbr', 'store_nbr'])[['unit_sales', 'price']].mean().reset_index()

avg_data.rename(columns={
    'unit_sales': 'avg_unit_sales',
    'price': 'avg_price'
}, inplace=True)

prod_data = data[['item_nbr', 'DESCRICAO', 'SECAO', 'GRUPO', 'SUBGRUPO']]
prod_data.drop_duplicates(inplace=True)

prod_store_data = prod_data.merge(avg_data, on='item_nbr')

# prod_store_data.to_csv('results/prod_store_data.csv', index=False)

In [831]:
def get_avg_sales(df):
    df['date'] = pd.to_datetime(df['date'])
    df['day'] = df['date'].dt.day.astype(np.int8)
    df['week'] = df['date'].dt.week.astype(np.int8)
    df['month'] = df['date'].dt.month.astype(np.int8)
    df['year'] = df['date'].dt.year
    df['year'] = (df['year'] - df['year'].min()).astype(np.int8)
    df['wom'] = df['day'].apply(lambda x: ceil(x/7)).astype(np.int8)
    df['dow'] = df['date'].dt.dayofweek.astype(np.int8) 
    df['weekend'] = (df['dow']>=5).astype(np.int8)

    dow_dict = {
        0: 'Segunda',
        1: 'Terça',
        2: 'Quarta',
        3: 'Quinta',
        4: 'Sexta',
        5: 'Sábado',
        6: 'Domingo'
    }

    df_dow = df.groupby(
        ['item_nbr', 'store_nbr', 'dow']
    )['unit_sales'].mean().reset_index()

    df_dow['dow'] = df_dow['dow'].apply(lambda x: dow_dict[x])
    df_dow.rename(
        columns={
            'unit_sales': 'avg_dow_sales'
        },
    inplace=True)
    
    df_wom = df.groupby(
        ['item_nbr', 'store_nbr', 'wom']
    )['unit_sales'].mean().reset_index()

    df_wom.rename(
        columns={
            'unit_sales': 'avg_wom_sales'
        },
    inplace=True)
    
    df_month = df.groupby(
        ['item_nbr', 'store_nbr', 'month']
    )['unit_sales'].mean().reset_index()

    df_month.rename(
        columns={
            'unit_sales': 'avg_monthly_sales'
        },
    inplace=True)
    
    return df_dow, df_wom, df_month

In [834]:
df_dow, df_wom, df_month = get_avg_sales(data)
# df_dow.to_csv('results/avg_dow_sales.csv', index=False)
# df_wom.to_csv('results/avg_wom_sales.csv', index=False)
# df_month.to_csv('results/avg_monthly_sales.csv', index=False)

---

In [728]:
data['date'] = pd.to_datetime(data['date'])
df_true_x_pred['date'] = pd.to_datetime(df_true_x_pred['date'])

In [770]:
EXPERIMENT_PERIOD = 7

input_complexity_weeks = {
    'low': 12,
    'high': 52
}

def get_last_year_date(date, n_years):
    d = date.strftime('%Y-%m-%d')
    l = d.split('-')
    d = '-'.join([str(int(l[0]) - n_years)] + l[1:])
    
    return datetime.strptime(d, '%Y-%m-%d')

In [847]:
# Generating experiments data
for i in tqdm(
    range(len(df_experiments)),
    bar_format='{l_bar}{bar:30}{r_bar}{bar:-30b}'
):
    experiment = df_experiments.iloc[i]
    experiment_id = experiment['id']
    item_nbr, store_nbr = experiment['item_nbr'], experiment['store_nbr']
    input_complexity = experiment['input_complexity']

    item_mask = data['item_nbr'] == item_nbr
    store_mask = data['store_nbr'] == store_nbr

    df = data[item_mask & store_mask]

    item_mask = df_true_x_pred['item_nbr'] == item_nbr
    store_mask = df_true_x_pred['store_nbr'] == store_nbr

    df_pred = df_true_x_pred[item_mask & store_mask]

    start_date = max([df_pred['date'].min(), datetime(2019, 1, 10)])
    end_date = min([df_pred['date'].max(), datetime(2019, 12, 10)])
    periods = (end_date - start_date).days

    date_range = pd.date_range(start_date, periods=periods).tolist()

    idxs = np.arange(0, len(date_range) - EXPERIMENT_PERIOD)
    idx = random.choice(idxs)
    experiment_date_range = date_range[idx:idx+EXPERIMENT_PERIOD]

    n_years = 1
    d = experiment_date_range[0]
    last_years_experiment_date_range = {}
    while d > datetime(2018, 1, 1):
        d = get_last_year_date(experiment_date_range[0], n_years)
        last_years_experiment_date_range[n_years] = pd.date_range(
            d, periods=EXPERIMENT_PERIOD
        ).tolist()
        n_years += 1

    n_weeks = input_complexity_weeks[input_complexity]
    df_ts = df[df['date'].isin(
        pd.date_range(
            experiment_date_range[0] - timedelta(weeks=n_weeks),
            periods=int(n_weeks*7)
        ).tolist()
    )][['date', 'unit_sales']]
    df_ts['experiment_id'] = experiment_id

    df_pred_to_predict = df_pred[df_pred['date'].isin(experiment_date_range)]
    df_to_predict = df[df['date'].isin(experiment_date_range)]

    df_pred_to_predict = df_pred_to_predict.merge(
        df_to_predict.drop(columns='unit_sales'), 
        on='date'
    )[['date', 'unit_sales', 'unit_sales_pred', 'price']]

    df_pred_to_predict['experiment_id'] = experiment_id

    df_hist = pd.DataFrame()
    for n_year, date_list in last_years_experiment_date_range.items():
        df_ = df[df['date'].isin(date_list)][
            ['date', 'unit_sales', 'price']
        ]
        df_['years_ago'] = n_year
        df_hist = pd.concat([df_hist, df_])

    df_hist['experiment_id'] = experiment_id

    df_pred_to_predict.to_csv(
        f'results/experiment_data/current/{str(experiment_id)}_data.csv',
        index=False
    )

    df_hist.to_csv(
        f'results/experiment_data/history/{str(experiment_id)}_data.csv',
        index=False
    )
    
    df_ts.to_csv(
        f'results/experiment_data/history/time_series/{str(experiment_id)}_data.csv',
        index=False
    )

100%|██████████████████████████████| 1210/1210 [00:31<00:00, 37.98it/s]


In [848]:
current_path = 'results/experiment_data/current'

df_current = pd.DataFrame()
for file in os.listdir(current_path):
    df_current = pd.concat([
        df_current,
        pd.read_csv(os.path.join(current_path, file))
    ])

In [849]:
history_path = 'results/experiment_data/history'

df_history = pd.DataFrame()
for file in os.listdir(history_path):
    try:
        df_history = pd.concat([
            df_history,
            pd.read_csv(os.path.join(history_path, file))
        ])
    except Exception as e:
        print(e)

[Errno 21] Is a directory: 'results/experiment_data/history/time_series'
[Errno 21] Is a directory: 'results/experiment_data/history/.ipynb_checkpoints'


In [850]:
history_ts_path = 'results/experiment_data/history/time_series'

df_history_ts = pd.DataFrame()
for file in os.listdir(history_ts_path):
    df_history_ts = pd.concat([
        df_history_ts,
        pd.read_csv(os.path.join(history_ts_path, file))
    ])

In [851]:
# df_current.to_csv('results/experiments_context.csv', index=False)
# df_history.to_csv('results/experiments_history.csv', index=False)
# df_history_ts.to_csv('results/experiments_history_ts.csv', index=False)